SVM on MNIST dataset 

In [3]:
%pip freeze | findstr "numpy scipy pandas libsvm"


libsvm==3.23.0.4
libsvm-official==3.35.0
numpy==1.22.4
pandas==1.1.5
scipy==1.7.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from libsvm.svmutil import svm_train, svm_predict, svm_problem, svm_parameter


X_train = pd.read_csv('./data/X_train.csv', header=None).values.tolist()
X_test = pd.read_csv('./data/X_test.csv', header=None).values.tolist()
y_train = pd.read_csv('./data/Y_train.csv', header=None).squeeze().tolist()
y_test = pd.read_csv('./data/Y_test.csv', header=None).squeeze().tolist()


In [ ]:
# kernel：0 = linear, 1 = polynomial, 2 = RBF
kernel_names = {0: "linear", 1: "polynomial", 2: "rbf"}
kernel_results = {}

for k in [0, 1, 2]:
    prob = svm_problem(y_train, X_train)
    if k == 1:
        param = svm_parameter('-s 0 -t 1 -d 3 -c 1')
    else:
        param = svm_parameter(f'-s 0 -t {k} -c 1')
    model = svm_train(prob, param)
    pred_label, acc, _ = svm_predict(y_test, X_test, model, options='-q')
    kernel_results[kernel_names[k]] = acc[0]

print("Kernel results: {}%".format(), kernel_results)

In [5]:
def mixed_kernel(X, Y, gamma=0.5):
    X = np.array(X)
    Y = np.array(Y)

    linear_part = X @ Y.T
    X_norm = np.sum(X ** 2, axis=1).reshape(-1, 1)
    Y_norm = np.sum(Y ** 2, axis=1).reshape(1, -1)
    rbf_part = np.exp(-gamma * (X_norm + Y_norm - 2 * np.dot(X, Y.T)))

    return  linear_part +  rbf_part


def add_index_column(K):
    n = K.shape[0]
    index = np.arange(1, n + 1, dtype=int).reshape(-1, 1)
    return np.hstack((index, K))



In [ ]:
def cross_validate_custom_kernel(K_full, y, C, gamma, num_folds=5):
    n = len(y)
    indices = list(range(n))
    np.random.seed(42)
    np.random.shuffle(indices)

    fold_size = n // num_folds
    folds = [indices[i * fold_size:(i + 1) * fold_size] for i in range(num_folds)]
    if n % num_folds != 0:
        for i in range(n % num_folds):
            folds[i].append(indices[num_folds * fold_size + i])

    acc_list = []

    for i in range(num_folds):
        val_idx = folds[i]
        train_idx = [j for j in indices if j not in val_idx]

        K_train = K_full[np.ix_(train_idx, train_idx)]
        K_val = K_full[np.ix_(val_idx, train_idx)]

        K_train_indexed = add_index_column(K_train)
        K_val_indexed = add_index_column(K_val)

        y_train = [y[j] for j in train_idx]
        y_val = [y[j] for j in val_idx]

        # prob = svm_problem(y_train, K_train_indexed.tolist())
        # param = svm_parameter(f'-t 4 -c {C} -q')
        # model = svm_train(prob, param)
        model = svm_train(y_train, K_train_indexed.tolist(), f'-t 4 -c {C} -g {gamma} -q ')

        _, acc, _ = svm_predict(y_val, K_val_indexed.tolist(), model, options='-q')
        acc_list.append(acc[0])

    return np.mean(acc_list)


def grid_search(X_train, y_train, X_test, y_test, kernel_type, param_grid):
    best_acc = 0
    best_params = None

    combinations = []

    if kernel_type == 'linear':
        for C in param_grid['C']:
            combinations.append([C])

    elif kernel_type == 'polynomial':
        for d in param_grid['degree']:
            for C in param_grid['C']:
                for g in param_grid['gamma']:
                    combinations.append([d, C, g])

    elif kernel_type == 'rbf':
        for g in param_grid['gamma']:
            for C in param_grid['C']:
                combinations.append([g, C])

    elif kernel_type == 'mixed':
        for C in param_grid['C']:
            for gamma in param_grid['gamma']:
                combinations.append([C, gamma])

    else:
        raise ValueError("Unsupported kernel type.")

    for params in combinations:
        if kernel_type == 'linear':
            C = params[0]
            param_str = f'-s 0 -t 0 -c {C} -v 5'
            prob = svm_problem(y_train, X_train)
            param = svm_parameter(param_str)
            acc = svm_train(prob, param, )  # acc is average CV accuracy
            print(f"Linear kernel: C={C}, acc={acc}")

        elif kernel_type == 'polynomial':
            degree, C, gamma = params
            param_str = f'-s 0 -t 1 -d {degree} -c {C} -g {gamma} -v 5'
            prob = svm_problem(y_train, X_train)
            param = svm_parameter(param_str)
            acc = svm_train(prob, param)
            print(f"Polynomial kernel: degree={degree}, C={C}, gamma={gamma}, acc={acc}")

        elif kernel_type == 'rbf':
            gamma, C = params
            param_str = f'-s 0 -t 2 -g {gamma} -c {C} -v 5'
            prob = svm_problem(y_train, X_train)
            param = svm_parameter(param_str)
            acc = svm_train(prob, param)
            print(f"RBF kernel: gamma={gamma}, C={C}, acc={acc}")

        elif kernel_type == 'mixed':
            C, gamma = params

            K_full = mixed_kernel(X_train, X_train, gamma)
            acc = cross_validate_custom_kernel(K_full, y_train, C, gamma)
            print("mixed kernel: C={}, gamma={}, acc={}".format(C, gamma, acc))

        else:
            continue

        if acc > best_acc:
            best_acc = acc
            best_params = params

    return best_params, best_acc



In [9]:

param_grid = {
    'linear': {'C': [0.1, 1, 10]},
    'polynomial': {'degree': [2, 3], 'C': [0.1, 1, 10], 'gamma': [0.01, 0.1]},
    'rbf': {'gamma': [0.01, 0.1], 'C': [0.1, 1, 10]}

}

kernel_results = {}

for kernel_type in param_grid.keys():
    best_params, best_acc = grid_search(X_train, y_train, X_test, y_test, kernel_type, param_grid[kernel_type])
    kernel_results[kernel_type] = (best_params, best_acc)

print("{:<12} | {:<25} | {:<10}".format("Kernel", "Best Parameters", "Accuracy"))
print("-" * 55)
for kernel_type, (best_params, best_acc) in kernel_results.items():
    print(f"{kernel_type:<12} | {best_params} | {best_acc:.4f}")



Cross Validation Accuracy = 96.96%
Linear kernel: C=0.1, acc=96.96000000000001
Cross Validation Accuracy = 96.3%
Linear kernel: C=1, acc=96.3
Cross Validation Accuracy = 96.1%
Linear kernel: C=10, acc=96.1
Cross Validation Accuracy = 94.78%
Polynomial kernel: degree=2, C=0.1, gamma=0.01, acc=94.78
Cross Validation Accuracy = 98.18%
Polynomial kernel: degree=2, C=0.1, gamma=0.1, acc=98.18
Cross Validation Accuracy = 97.66%
Polynomial kernel: degree=2, C=1, gamma=0.01, acc=97.66
Cross Validation Accuracy = 98.2%
Polynomial kernel: degree=2, C=1, gamma=0.1, acc=98.2
Cross Validation Accuracy = 98.04%
Polynomial kernel: degree=2, C=10, gamma=0.01, acc=98.04
Cross Validation Accuracy = 97.98%
Polynomial kernel: degree=2, C=10, gamma=0.1, acc=97.98
Cross Validation Accuracy = 90.14%
Polynomial kernel: degree=3, C=0.1, gamma=0.01, acc=90.14
Cross Validation Accuracy = 97.92%
Polynomial kernel: degree=3, C=0.1, gamma=0.1, acc=97.92
Cross Validation Accuracy = 96.54%
Polynomial kernel: degree=3

In [7]:
param_grid = {
    # 'linear': {'C': [0.1, 1, 10]},
    # 'polynomial': {'degree': [2, 3], 'C': [0.1, 1, 10], 'gamma': [0.01, 0.1]},
    # 'rbf': {'gamma': [0.01, 0.1], 'C': [0.1, 1, 10]},
    'mixed': {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1]}

}

kernel_results = {}

for kernel_type in param_grid.keys():
    best_params, best_acc = grid_search(X_train, y_train, X_test, y_test, kernel_type, param_grid[kernel_type])
    kernel_results[kernel_type] = (best_params, best_acc)


print("{:<12} | {:<25} | {:<10}".format("Kernel", "Best Parameters", "Accuracy"))
print("-" * 55)
for kernel_type, (best_params, best_acc) in kernel_results.items():
    print(f"{kernel_type:<12} | {best_params} | {best_acc:.4f}")


mixed kernel: C=0.1, gamma=0.01, acc=97.16
mixed kernel: C=0.1, gamma=0.1, acc=97.16
mixed kernel: C=1, gamma=0.01, acc=96.61999999999999
mixed kernel: C=1, gamma=0.1, acc=96.72
mixed kernel: C=10, gamma=0.01, acc=96.61999999999999
mixed kernel: C=10, gamma=0.1, acc=96.72
Kernel       | Best Parameters           | Accuracy  
-------------------------------------------------------
mixed        | [0.1, 0.01] | 97.1600


In [ ]:
# %pip install libsvm==3.23.0.4


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install numpy==1.19.0
# %pip install pandas==1.1.5

  Using cached numpy-1.19.0.zip (7.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Running from numpy source directory.
      <string>:460: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Traceback (most recent call last):
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 175, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
        File "C:\Users\yuc

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
aiohappyeyeballs             2.4.6
aiohttp                      3.11.12
aiosignal                    1.3.2
asttokens                    3.0.0
astunparse                   1.6.3
async-timeout                5.0.1
attrs                        25.1.0
blinker                      1.9.0
Brotli                       1.1.0
cached-property              1.5.2
cachetools                   5.5.2
certifi                      2025.1.31
cffi                         1.17.1
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.0
cryptography                 39.0.0
cycler                       0.12.1
debugpy                      1.6.7
decorator                    5.2.0
exceptiongroup               1.2.2
executing                    2.1.0
flatbuffers                  25.2.10
